In [1]:
!uv run python -m pytest "test.py::test_fsdp_with_itself" -q

.F.F                                                                     [100%]
=================================== FAILURES ===================================
___________________ test_fsdp_with_itself[float32-bfloat16] ____________________

param_dtype = 'bfloat16', reduce_dtype = 'float32'

    @pytest.mark.parametrize("param_dtype", [None, "bfloat16"])
    @pytest.mark.parametrize("reduce_dtype", ["float32", "bfloat16"])
    def test_fsdp_with_itself(
        param_dtype: Literal["bfloat16", "float32"] | None,
        reduce_dtype: Literal["bfloat16", "float32"] | None,
    ):
>       run_distributed_test(
            _test_fsdp_with_itself,
            param_dtype=param_dtype,
            reduce_dtype=reduce_dtype,
            world_size=2,
        )

test.py:116: 
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 
test.py:82: in run_distributed_test
    mp.start_processes(  # type: ignore[attr-defined,no-untyped-call]
.venv/lib/python3.12/site-packag

**как можно заметить fsdp2 сам против себя не выдает стабильные результаты** так что поменял тесты чтобы они принимали погрешность до 1e-3

In [2]:
!uv run python -m pytest "test.py::test_fsdp" -q

....                                                                     [100%]
=============================== warnings summary ===============================
.venv/lib/python3.12/site-packages/torch/jit/_script.py:362: 14 warnings
  /workspace/efdl_hw/week05_fsdp/homework/.venv/lib/python3.12/site-packages/torch/jit/_script.py:362: DeprecationWarning: `torch.jit.script_method` is deprecated. Please switch to `torch.compile` or `torch.export`.
    warnings.warn(

-- Docs: https://docs.pytest.org/en/stable/how-to/capture-warnings.html
4 passed, 14 warnings in 43.00s


In [3]:
!uv run torchrun --standalone --nproc-per-node=2 train.py --fsdp fsdp2 --num-steps-to-profile 3 --snapshots-dir snapshots/fsdp2 --traces-dir traces/fsdp2 --model 1b --seq-len 1024

W0227 19:16:16.777000 128690 torch/distributed/run.py:852] 
W0227 19:16:16.777000 128690 torch/distributed/run.py:852] *****************************************
W0227 19:16:16.777000 128690 torch/distributed/run.py:852] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W0227 19:16:16.777000 128690 torch/distributed/run.py:852] *****************************************


2026-02-27 19:16:22,734 - __main__ - INFO - model size: 1498482688 parameters.
2026-02-27 19:16:22,751 - __main__ - INFO - model size: 1498482688 parameters.
2026-02-27 19:16:22,796 - root - INFO - Loading tokenizer from tokenizer.json
2026-02-27 19:16:22,798 - root - INFO - Preparing c4_test dataset from ./c4_test
2026-02-27 19:16:22,818 - root - INFO - Loading tokenizer from tokenizer.json
2026-02-27 19:16:22,820 - root - INFO - Preparing c4_test dataset from ./c4_test
/workspace/efdl_hw/week05_fsdp/homework/.venv/lib/python3.12/site-packages/torch/profiler/profiler.py:217: UserWarning: Warning: Profiler clears events at the end of each cycle.Only events from the current cycle will be reported.To keep events across cycles, set acc_events=True.
  _warn_once(
/workspace/efdl_hw/week05_fsdp/homework/.venv/lib/python3.12/site-packages/torch/profiler/profiler.py:217: UserWarning: Warning: Profiler clears events at the end of each cycle.Only events from the current cycle will be reported.T

In [4]:
!uv run torchrun --standalone --nproc-per-node=2 train.py --fsdp effdl --num-steps-to-profile 3 --snapshots-dir snapshots/effdl --traces-dir traces/effdl  --model 1b --seq-len 1024

W0227 19:18:43.046000 129002 torch/distributed/run.py:852] 
W0227 19:18:43.046000 129002 torch/distributed/run.py:852] *****************************************
W0227 19:18:43.046000 129002 torch/distributed/run.py:852] Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
W0227 19:18:43.046000 129002 torch/distributed/run.py:852] *****************************************
2026-02-27 19:18:48,851 - __main__ - INFO - model size: 1498482688 parameters.
2026-02-27 19:18:48,853 - __main__ - INFO - model size: 1498482688 parameters.
2026-02-27 19:18:48,910 - root - INFO - Loading tokenizer from tokenizer.json
2026-02-27 19:18:48,913 - root - INFO - Preparing c4_test dataset from ./c4_test
2026-02-27 19:18:48,913 - root - INFO - Loading tokenizer from tokenizer.json
2026-02-27 19:18:48,915 - root - INFO - Preparing c4_test dataset from ./c

<img src="mem_effdl.png" alt="Memory comparison" width="600"/>

<img src="mem_fsdp2.png" alt="Memory comparison" width="600"/>

как видно мемори снепшоты получаются очень похожими

<img src="prof_effdl.png" alt="Memory comparison" width="600"/>

<img src="prof_fsdp2.png" alt="Memory comparison" width="600"/>

также можно увидеть что профиль почти совпадает